In [5]:
import duckdb

import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils 

## Join two sources into one table

In [6]:

nationalites = utils.DBtable_to_df('../integration.duckdb', 'nationalitiesClean_Madrid')
households = utils.DBtable_to_df('../integration.duckdb','householdClean_Madrid')

result = households.merge(nationalites, how='inner', on=["Madrid_section","Year"])

utils.df_to_DBtable('../integration.duckdb',result, 'integratedTable')

### Test

In [7]:
# check if all tables in integration.duckdb are created
utils.get_tables('../integration.duckdb')

0    nationalitiesClean_Madrid
1        householdClean_Madrid
2              integratedTable
Name: table_name, dtype: object

## Removing redundacies from integrated table

### Removing columns with zeros in all rows

In [41]:
integratedDF = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')
for (columnName, columnData) in integratedDF.items():
    if(integratedDF[columnName] == 0).all():
        integratedDF.drop(columnName, axis=1, inplace=True)
        print(columnName)
display(integratedDF)


Brunei
Maldivas
Santa_Sede
Swazilandia
Vanuatu


,Madrid_section,population,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,079601001,559.0,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,...,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,079601002,471.0,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,...,0.0,0.0,0.0,3.0,0.0,13.0,0.0,0.0,0.0,0.0
2,079601003,855.0,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,...,0.0,7.0,0.0,4.0,0.0,22.0,1.0,0.0,0.0,0.0
3,079601004,593.0,99.0,95.0,55.0,20.0,9.0,1.0,88.0,71.0,...,0.0,1.0,1.0,2.0,0.0,11.0,0.0,0.0,0.0,0.0
4,079601006,882.0,142.0,174.0,75.0,42.0,12.0,1.0,151.0,80.0,...,0.0,6.0,0.0,2.0,0.0,13.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,079621029,514.0,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
7325,079621030,561.0,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,...,0.0,8.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0
7326,079621031,988.0,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,...,0.0,8.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0
7327,079621032,683.0,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,...,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0


### Removing aggregated columns 'suma de categorías de nacionalidad' and 'population'

In [42]:
integratedDF.drop(['suma de categorías de nacionalidad', 'population'], axis=1, inplace=True)
display(integratedDF)


,Madrid_section,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,079601001,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,15.0,...,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,079601002,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,11.0,...,0.0,0.0,0.0,3.0,0.0,13.0,0.0,0.0,0.0,0.0
2,079601003,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,23.0,...,0.0,7.0,0.0,4.0,0.0,22.0,1.0,0.0,0.0,0.0
3,079601004,99.0,95.0,55.0,20.0,9.0,1.0,88.0,71.0,23.0,...,0.0,1.0,1.0,2.0,0.0,11.0,0.0,0.0,0.0,0.0
4,079601006,142.0,174.0,75.0,42.0,12.0,1.0,151.0,80.0,33.0,...,0.0,6.0,0.0,2.0,0.0,13.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,079621029,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,55.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
7325,079621030,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,46.0,...,0.0,8.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0
7326,079621031,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,162.0,...,0.0,8.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0
7327,079621032,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,68.0,...,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0


### Checking if there is section code with all zeros in remaining columns

In [43]:

display(integratedDF[integratedDF.columns[1:]].loc[(integratedDF[integratedDF.columns[1:]]== 0).all(axis=1)])
# none are found



,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,two_adults_and_two_minors,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


### Updating integrated table 

In [44]:
utils.df_to_DBtable('../integration.duckdb',integratedDF, 'integratedTable')

## Saving copy of integrated table to analysis folder, together with utils needed for analysis

In [48]:
import shutil

shutil.copy(os.path.join("../integration.duckdb"), os.path.join("../analysis/"))
shutil.copy(os.path.join("../utils.py"), os.path.join("../analysis/"))

print('Copied')

Copied
